# Tables properties

In this section, I'll describe some common tasks associated with posgres and their solutions.

The following cell creates all needed for examples in this page.

In [20]:
docker run --rm -d\
    -e POSTGRES_PASSWORD=postgres \
    --name tables_properties_example \
    postgres:15.4 &> /dev/null
sleep 5

**Stop container!** After playing with examples, you should turn it off with the following command.

In [35]:
docker stop tables_properties_example &> /dev/null

## Columns information

There is an `information_schema.columns` table that contains information about the columns from the tables in the database.

---

The following cell shows a raw view of that table. It's not empty because PostgreSQL, by default, has some system tables.

In [28]:
docker exec -i tables_properties_example psql -U postgres << EOF
SELECT *
FROM information_schema.columns
LIMIT 10;
EOF

 table_catalog | table_schema |  table_name   | column_name | ordinal_position |              column_default               | is_nullable | data_type | character_maximum_length | character_octet_length | numeric_precision | numeric_precision_radix | numeric_scale | datetime_precision | interval_type | interval_precision | character_set_catalog | character_set_schema | character_set_name | collation_catalog | collation_schema | collation_name | domain_catalog | domain_schema | domain_name | udt_catalog | udt_schema | udt_name | scope_catalog | scope_schema | scope_name | maximum_cardinality | dtd_identifier | is_self_referencing | is_identity | identity_generation | identity_start | identity_increment | identity_maximum | identity_minimum | identity_cycle | is_generated | generation_expression | is_updatable 
---------------+--------------+---------------+-------------+------------------+-------------------------------------------+-------------+-----------+--------------------------+----

The following cell shows how to check the data types of the just created table:

In [34]:
docker exec -i tables_properties_example psql -U postgres << EOF
CREATE TABLE dtype_example (
    id INTEGER,
    text TEXT
);

SELECT column_name, data_type FROM information_schema.columns WHERE table_name='dtype_example';

DROP TABLE dtype_example;
EOF

CREATE TABLE
 column_name | data_type 
-------------+-----------
 id          | integer
 text        | text
(2 rows)

DROP TABLE


## Count raws

To get counts of observations in table you can use command: `SELECT COUNT(*) FROM <table_name>`.

---

The example in the following cell shows counting the result of the `generate_series(1, 1000000)` function, which generates a series of values with the specified length.

In [13]:
docker exec -i tables_properties_example psql -U postgres << EOF
SELECT COUNT(*) FROM generate_series(1, 1000000);
EOF

  count  
---------
 1000000
(1 row)



## List tables

It's a common task to list all available tables for the current database. So in this section I want to mention some options.

The following cell prepares a set of tables that we'll use as examples.

In [21]:
docker exec -i tables_properties_example psql -U postgres << EOF
DO \$\$ 
DECLARE
    num_tables INT := 5; -- specify the number of tables
BEGIN
    FOR i IN 1..num_tables LOOP
        EXECUTE format('CREATE TABLE list_tables_%s (id SERIAL PRIMARY KEY, name TEXT);', i);
    END LOOP;
END \$\$;
EOF

DO


### `\dt`

Very simple method, but it doesn't always work.

In [22]:
docker exec -i tables_properties_example psql -U postgres << EOF
\dt
EOF

             List of relations
 Schema |     Name      | Type  |  Owner   
--------+---------------+-------+----------
 public | list_tables_1 | table | postgres
 public | list_tables_2 | table | postgres
 public | list_tables_3 | table | postgres
 public | list_tables_4 | table | postgres
 public | list_tables_5 | table | postgres
(5 rows)



### pg_catalog.pg_tables table

This is a service table that contains the tables available for this base.

---

Folowing cell just shows whole `pg_catalog.pg_tables`.

In [26]:
docker exec -i tables_properties_example psql -U postgres << EOF
SELECT * FROM pg_catalog.pg_tables;
EOF

     schemaname     |        tablename         | tableowner | tablespace | hasindexes | hasrules | hastriggers | rowsecurity 
--------------------+--------------------------+------------+------------+------------+----------+-------------+-------------
 public             | list_tables_1            | postgres   |            | t          | f        | f           | f
 public             | list_tables_2            | postgres   |            | t          | f        | f           | f
 public             | list_tables_3            | postgres   |            | t          | f        | f           | f
 public             | list_tables_4            | postgres   |            | t          | f        | f           | f
 public             | list_tables_5            | postgres   |            | t          | f        | f           | f
 pg_catalog         | pg_statistic             | postgres   |            | t          | f        | f           | f
 pg_catalog         | pg_type                  | postgres 

But we can impose restrictions, for example, on the `schemaname` column and get only those tables we are interested in.

In [25]:
docker exec -i tables_properties_example psql -U postgres << EOF
SELECT * FROM pg_catalog.pg_tables WHERE schemaname='public';
EOF

 schemaname |   tablename   | tableowner | tablespace | hasindexes | hasrules | hastriggers | rowsecurity 
------------+---------------+------------+------------+------------+----------+-------------+-------------
 public     | list_tables_1 | postgres   |            | t          | f        | f           | f
 public     | list_tables_2 | postgres   |            | t          | f        | f           | f
 public     | list_tables_3 | postgres   |            | t          | f        | f           | f
 public     | list_tables_4 | postgres   |            | t          | f        | f           | f
 public     | list_tables_5 | postgres   |            | t          | f        | f           | f
(5 rows)

